# Post-process morphotope IDs to ensure IDs allocated to noise are unique per contiguity group

In [1]:
import geopandas as gpd

from libpysal import graph

In [20]:
v = 'v10'

region_hulls = gpd.read_parquet('/data/uscuni-ulce/regions/cadastre_regions_hull.parquet')

In [21]:
region_hulls

,convex_hull
labels,
4,"POLYGON ((4122200 3045600, 4121500 3045900, 40..."
10,"POLYGON ((4068900 3057300, 4051000 3064300, 40..."
132,"POLYGON ((4053100 3049800, 4050900 3053000, 40..."
134,"POLYGON ((4052500 3093600, 4052200 3093700, 40..."
286,"POLYGON ((4060600 3103900, 4057000 3104200, 40..."
...,...
149997,"POLYGON ((5253200 3642300, 5245600 3647000, 52..."
150044,"POLYGON ((5265700 3578300, 5263400 3578400, 52..."
151676,"POLYGON ((5285400 3549200, 5284800 3549900, 52..."


In [113]:
%%time

for region in region_hulls.index:
    df = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/clusters/clusters_{region}_{v}.pq')
    w = graph.read_parquet(f'/data/uscuni-ulce/processed_data/neigh_graphs/tessellation_graph_{region}.parquet')
    noise = df[df.morph.str.endswith('-1')]
    noise_labels = w.subgraph(noise.index).component_labels
    df['unique_morph'] = df['morph']
    df.loc[noise.index, 'unique_morph'] = df.loc[noise.index, 'unique_morph'].str[:-1] + (noise_labels + 1).astype('str')
    df.to_parquet(f'/data/uscuni-ulce/processed_data/clusters/clusters_{region}_{v}.pq')

CPU times: user 6min 5s, sys: 38.3 s, total: 6min 43s
Wall time: 6min 26s


In [80]:
region = 69333

In [183]:
%%time

for region in region_hulls.index:
    buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/clusters/clusters_{region}_{v}.pq', columns=['morph', 'unique_morph', 'geometry'])
    buildings_noise = buildings[buildings.morph.str.endswith('-1')]
    join_data = buildings_noise.representative_point().to_frame('geometry').dissolve(buildings_noise.unique_morph).representative_point()
    
    mtopes = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/morphotope_clusters/{v}/{region}_clusters.pq').reset_index()
    mtopes_noise = mtopes[mtopes.morph.str.endswith('-1')].copy()
    joined = mtopes_noise.reset_index().sjoin(join_data.to_frame(), how='left')
    cleaned = joined.groupby('index', as_index=False).first()
    mtopes['unique_morph'] = mtopes.morph.copy()
    mtopes.loc[cleaned['index'], 'unique_morph'] = cleaned.unique_morph.values
    mtopes.to_parquet(f'/data/uscuni-ulce/processed_data/morphotope_clusters/{v}/{region}_clusters.pq')

CPU times: user 4min 11s, sys: 52.9 s, total: 5min 4s
Wall time: 5min 3s
